# RN-B — Coherencia fotométrica temprana en 47 Tuc: curva R(g)

**Propósito.** Construir la figura de R(g) y el panel de mitigaciones **a partir de métricas ya exportadas** por `dp1_47tuc_analysis.ipynb`.

**Entradas.** `data/47tuc_dp1/rnB_metrics.json`

**Salidas.**
- `notebooks/47tuc/figs/rnB_R_of_g.png`
- `notebooks/47tuc/figs/rnB_systematics_panel.png`

In [ ]:
import os, json
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

In [ ]:
metrics_path = 'data/47tuc_dp1/rnB_metrics.json'
assert os.path.exists(metrics_path), 'Falta rnB_metrics.json. Corre primero dp1_47tuc_analysis.ipynb para exportarlo.'
with open(metrics_path,'r') as f:
    M = json.load(f)
df = pd.DataFrame(M['bins'])  # espera columnas: g_center, R, ci_low, ci_high, N
df = df.sort_values('g_center')
print('Bins cargados:', len(df))

In [ ]:
plt.figure(figsize=(7,5))
x = df['g_center'].to_numpy(); R = df['R'].to_numpy()
ci_low = df['ci_low'].to_numpy(); ci_high = df['ci_high'].to_numpy()
yerr = np.vstack([R-ci_low, ci_high-R])
plt.errorbar(x, R, yerr=yerr, fmt='o', ms=4, capsize=3, label='R(g) ± IC95%')
plt.gca().invert_xaxis()  # opcional: magnitud crece a la derecha si prefieres
plt.xlabel('g (mag)'); plt.ylabel('R = sigma_obs_perp / median(sigma_rep_perp)')
plt.title('47 Tuc — DP1 (ComCam)')
plt.grid(alpha=0.3); plt.tight_layout()
from pathlib import Path
NB_DIR = Path.cwd()
FIG_DIR = NB_DIR / "figs"
FIG_DIR.mkdir(parents=True, exist_ok=True)

outfig = FIG_DIR / "rnB_R_of_g.png"             # primer gráfico
plt.savefig(outfig, dpi=200)

outfig2 = FIG_DIR / "rnB_systematics_panel.png" # panel de sistemáticos
plt.savefig(outfig2, dpi=200)

In [ ]:
# Panel de sistemáticos: espera que M contenga una entrada 'systematics_example' con
# los valores para una ventana representante (p.ej. g≈19.5): raw, +5mmag, +10mmag, detrend.
if 'systematics_example' in M:
    S = M['systematics_example']
    labels = ['Raw','+5 mmag','+10 mmag','Detrend']
    vals = [S.get('R_raw',np.nan), S.get('R_5mmag',np.nan), S.get('R_10mmag',np.nan), S.get('R_detrend',np.nan)]
    plt.figure(figsize=(6,4))
    plt.bar(labels, vals, color=['#1f77b4','#2ca02c','#17becf','#ff7f0e'])
    plt.ylabel('R en ventana')
    plt.title('Mitigación de sistemáticos (ventana representativa)')
    plt.tight_layout()
    outfig2 = 'notebooks/47tuc/figs/rnB_systematics_panel.png'
    plt.savefig(outfig2, dpi=200)
    print('Figura guardada ->', outfig2)
else:
    print('Aviso: rnB_metrics.json no trae systematics_example; salta panel.')